# IMPORTS

In [20]:
import pandas as pd 
import seaborn as sns
import statsmodels as stsm
import numpy as np
import sklearn as skl
from pycaret.classification import *
from sklearn.metrics import mean_squared_error, r2_score # Para Cálculo de R2 e RMSE
from sklearn.model_selection import train_test_split #modelagem >:)

# VERIFICANDO O CSV WINE QUALITY

In [21]:
df = pd.read_csv('winequality-red.csv')
df.head(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [22]:
#Corrigindo espaço entre os nomes das colunas (afeta em sua chamada nas funções)
colunas_at = df.columns

colunas_novas = [col.strip().lower().replace(' ', '_').replace('.','_') for col in colunas_at]

df.columns = colunas_novas

df.columns

Index(['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar',
       'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density',
       'ph', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

Escolheremos a coluna 'quality' como variável alvo das regressões linear e múltipla.

In [23]:
#Treino será usado para o statsmodel
#Testes será usado para o preditivo do sklearn
df_treino, df_testes = train_test_split(df, test_size=0.3, random_state=42)

## HIPOTESE DA REGRESSÃO LINEAR

H0: O coeficiente entre qualidade e pH é zero

Ha: O coeficiente entre qualidade e ph é diferente de zero

In [24]:
#Usamos do statsmodels para com sua api, verificarmos a regressão linear

modelo = stsm.formula.api.ols('quality ~ ph', data = df_treino).fit()

print(f"p-valor para pH: {modelo.pvalues['ph']:.4f}")
print('Tabela dos coeficientes')
print(modelo.summary().tables[1]) #Aqui tem os coeficientes, p-valores e R2

p-valor para pH: 0.3695
Tabela dos coeficientes
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      6.0912      0.523     11.640      0.000       5.064       7.118
ph            -0.1416      0.158     -0.898      0.370      -0.451       0.168


In [25]:
predicoes_linear = modelo.predict(df_testes)

r2_simples = r2_score(df_testes['quality'], predicoes_linear)
rmse_simples = np.sqrt(mean_squared_error(df_testes['quality'], predicoes_linear))
                       
print(f'R-Squared (R2) no teste: {r2_simples:.4f}') #r2 reportado!
print(f'RMSE no teste: {rmse_simples:.4f}') #rmse aqui!

R-Squared (R2) no teste: 0.0031
RMSE no teste: 0.7950


## REGRESSÃO MÚLTIPLA

Para o modelo de regressão múltipla, usaremos a comparação entre quality e Ph, alcohol e volatile_acidity

In [ ]:
modelo_multiplo = stsm.formula.api.ols('quality ~ ph + alcohol + volatile_acidity', data = df_treino).fit()

print("Tabela de coeficientes (para interpretação da H0 de cada variável analisada).")

print(modelo_multiplo.summary().tables[1])

Tabela de coeficientes (para interpretação da H0 de cada variável analisada).
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            3.6631      0.445      8.238      0.000       2.791       4.536
ph                  -0.2779      0.137     -2.022      0.043      -0.548      -0.008
alcohol              0.3401      0.020     17.079      0.000       0.301       0.379
volatile_acidity    -1.2446      0.117    -10.678      0.000      -1.473      -1.016


In [27]:
#Predições a partir do teste
predicoes_reg_multp = modelo_multiplo.predict(df_testes)

r2_multiplo = r2_score(df_testes['quality'], predicoes_reg_multp)
rmse_multiplo = np.sqrt(mean_squared_error(df_testes['quality'], predicoes_reg_multp))

print(f'R-Squared (R²) no teste: {r2_multiplo:.4f}')
print(f'RMSE no teste: {rmse_multiplo:.4f}')


R-Squared (R²) no teste: 0.3066
RMSE no teste: 0.6630


## REGRESSÃO POLINOMIAL

In [29]:
#Para uma regressão de grau 2 (polinomial), precisamos usar uma função identidade
#e elevar a variável ao quadrado, indicando que deve ser um termo do modelo

modelo_polinomial = stsm.formula.api.ols('quality ~ alcohol + I(alcohol**2) + volatile_acidity', data = df_treino).fit()
print('Tabela dos coeficientes (Mostra o termo I(alcohol**2)):')
print(modelo_polinomial.summary().tables[1])

Tabela dos coeficientes (Mostra o termo I(alcohol**2)):
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -1.1230      1.708     -0.657      0.511      -4.475       2.229
alcohol              1.0731      0.315      3.409      0.001       0.455       1.691
I(alcohol ** 2)     -0.0341      0.014     -2.366      0.018      -0.062      -0.006
volatile_acidity    -1.3066      0.112    -11.664      0.000      -1.526      -1.087


In [31]:
#Prevendo a partir dos testes, padrão
predicoes_polinomiais = modelo_polinomial.predict(df_testes)

r2_poli = r2_score(df_testes['quality'], predicoes_polinomiais)
rmse_poli = np.sqrt(mean_squared_error(df_testes['quality'], predicoes_polinomiais))

print(f'R-Squared (R²) no Teste: {r2_poli:.4f}')
print(f'RMSE no Teste: {rmse_poli:.4f}')

R-Squared (R²) no Teste: 0.2895
RMSE no Teste: 0.6712
